## Preparing the data

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# change working directory to model folder
os.chdir("C:\\Users\\brian\\Desktop\\minor\\AI project\\model")

# save the data from the csv file and divide Pawpularity score by 100
train_data = pd.read_csv('petfinder-pawpularity-score/train.csv')
train_data['Pawpularity'] = train_data['Pawpularity']

# check if the data was read correctly and if there are NaN values
display(train_data.head(5))
print(f'NaN amount: {train_data.isnull().sum().sum()}')

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


NaN amount: 0


In [25]:
def load_images_from_folder(folder, data = [], size=128):
    """
    This function takes a folder in which the images are saved, a dataframe of tabular data and size.
    It loads in an image finds the corresponding name and Pawpularity score and saves those so that they are ordered.
    It returns an array of images, a list of labels and a list of names.
    """
    # create empty lists for the data
    images = []
    labels = []
    names = []
    
    # iterate over all images in the given folder
    for filename in os.listdir(folder):
        
        # extract the name from the image
        name = filename.strip('.jpg')
        
        # add the name to the list of names
        names.append(name)
        
        # use the name to find the corresponding pawpularity score and add it to the list of scores
        labels.append(data.loc[data['Id']== name]['Pawpularity'].item())
        
        # resize the image to the given size and add it to the list of images
        img = cv2.resize(cv2.imread(os.path.join(folder,filename)), (size, size))
        if img is not None:
            images.append(img)
            
    # return the array of images and lists of labels and names
    return np.array(images), labels, names

data_images, data_labels, data_names = load_images_from_folder('petfinder-pawpularity-score/train', data = train_data)

print(f'Images loaded: {data_images.shape}')

Images loaded: (9912, 64, 64, 3)


In [47]:
X = np.array(train_data[['Subject Focus', 'Eyes', 'Face', 'Near', "Action", 'Accessory','Group', 'Collage','Human', 'Occlusion', 'Info', 'Blur']])

In [50]:
# split the data into 67% training data and 33% testing data
image_train, image_test, label_train, label_test, X_train, X_test = train_test_split(data_images, data_labels, X, test_size=0.33, random_state=42)

In [30]:
# check if the labels match the names of the images
print('checking training data: ')
print(label_train[0])


checking training data: 
99
ec07599018885a44734a9ba27d306055


## Making the image model

In [31]:
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, preprocessing, utils
import tensorflow as tf

In [55]:
def train_and_evaluate(model, train_x, train_y, val_x, val_y, X_train, X_test, preprocess={}, epochs=20, augment={}):
    ''' Make the model to train and evaluate the data'''

    
    
    # compile the model using adam as optimizer with mean squared error as loss function
    model.compile(optimizer='adam',
              loss='MSE',
              metrics=['RootMeanSquaredError'])

    train_gen = preprocessing.image.ImageDataGenerator(preprocess, augment)
    train_gen.fit(train_x) 

    val_gen = preprocessing.image.ImageDataGenerator(**preprocess)
    val_gen.fit(val_x)

    # estimate the model using the compile 
    history = model.fit(train_gen.flow((train_x, X_train), train_y), epochs=epochs, 
                        validation_data=val_gen.flow((val_x, X_test), val_y))

    fig, axs = plt.subplots(1,2,figsize=(20,5)) 

    # plot the learning curves
    for i, metric in enumerate(['loss', 'RootMeanSquaredError']):
        axs[i].plot(history.history[metric])
        axs[i].plot(history.history['val_'+metric])
        axs[i].legend(['training', 'validation'], loc='best')

        axs[i].set_title('Model '+metric)
        axs[i].set_ylabel(metric)
        axs[i].set_xlabel('epoch')

    plt.show()



  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 208 steps, validate for 103 steps
Epoch 1/50
208/208 [==============================] - 4s 20ms/step - loss: 556.2905 - RootMeanSquaredError: 23.6003 - val_loss: 454.7082 - val_RootMeanSquaredError: 21.3271
Epoch 2/50
208/208 [==============================] - 3s 15ms/step - loss: 450.0549 - RootMeanSquaredError: 21.2209 - val_loss: 492.3135 - val_RootMeanSquaredError: 22.1777
Epoch 3/50
208/208 [==============================] - 3s 15ms/step - loss: 429.7593 - RootMeanSquaredError: 20.7377 - val_loss: 445.7257 - val_RootMeanSquaredError: 21.1204
Epoch 4/50
208/208 [==============================] - 3s 15ms/step - loss: 418.5653 - RootMeanSquaredError: 20.4668 - val_loss: 448.5471 - val_RootMeanSquaredError: 21.1777
Epoch 5/50
208/208 [==============================] - 3s 15ms/step - loss: 413.9299 - RootMeanSquaredError: 20.3539 - val_loss: 444.4228 - val_RootMeanSquaredError: 21.0819
Epoch 6/50
208/208 [=====================

Epoch 47/50
208/208 [==============================] - 3s 15ms/step - loss: 61.9427 - RootMeanSquaredError: 7.8701 - val_loss: 478.9791 - val_RootMeanSquaredError: 21.8949
Epoch 48/50
208/208 [==============================] - 3s 15ms/step - loss: 64.3381 - RootMeanSquaredError: 8.0196 - val_loss: 481.7901 - val_RootMeanSquaredError: 21.9625
Epoch 49/50
208/208 [==============================] - 3s 15ms/step - loss: 65.2635 - RootMeanSquaredError: 8.0765 - val_loss: 464.1243 - val_RootMeanSquaredError: 21.5534
Epoch 50/50
208/208 [==============================] - 3s 15ms/step - loss: 60.5025 - RootMeanSquaredError: 7.7843 - val_loss: 710.1149 - val_RootMeanSquaredError: 26.6794


In [61]:
# create the CNN model by adding the layers
model_img = tf.keras.Input(shape = image_train.shape[1:], name = 'IMG')

model_img_1 = tf.keras.layers.Conv2D(64,(3,3),activation='relu' ,padding='same', name = 'IMG_1')(model_img)
model_img_2 = tf.keras.layers.MaxPool2D((2,2))(model_img_1)

# second layer
model_img_3 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same')(model_img_2)
model_img_4 = tf.keras.layers.MaxPool2D((2,2))(model_img_3)
model_img_5 = tf.keras.layers.BatchNormalization()(model_img_4)

# third layer
model_img_6 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='same')(model_img_5)
model_img_7 = tf.keras.layers.MaxPool2D((2,2))(model_img_6)
model_img_8 = tf.keras.layers.BatchNormalization()(model_img_7)

# fourth layer
model_img_9 = tf.keras.layers.Dropout(0.1)(model_img_8)
model_img_10 = tf.keras.layers.Conv2D(512,(3,3),activation='relu',padding='same')(model_img_9)
model_img_11 = tf.keras.layers.MaxPool2D((2,2))(model_img_10)
model_img_12 = tf.keras.layers.BatchNormalization()(model_img_11)


model_img_13 = tf.keras.layers.Flatten()(model_img_12)

model_img_14 = tf.keras.layers.Dropout(0.4)(model_img_13)
model_img_15 = tf.keras.layers.Dense(512,activation='relu')(model_img_14)

model_img_16 = tf.keras.layers.Dropout(0.4)(model_img_15)
model_img_17 = tf.keras.layers.Dense(256,activation='relu')(model_img_16)

model_img_18 = tf.keras.layers.Dropout(0.4)(model_img_17)
model_img_19 = tf.keras.layers.Dense(128,activation='relu')(model_img_18)

model_img_20 = tf.keras.layers.Dropout(0.4)(model_img_19)


##### tabular
tabular = tf.keras.Input(shape = X.shape[1:], name = 'tabular') 
tabular1 = tf.keras.layers.Dense(30, activation='relu', name='tabular1')(tabular)
tabular2 = tf.keras.layers.Dense(50, activation='relu', name='tabular2')(tabular1)
tabular3 = tf.keras.layers.Dense(30, activation='relu', name='tabular3')(tabular2)
tabular4 = tf.keras.layers.Dense(10, activation='relu', name='tabular4')(tabular3)

model_concat = tf.keras.layers.Concatenate(name = 'connection', axis = 1)([tabular4, model_img_20])

model_fin = tf.keras.layers.Dense(1, activation = 'linear', name = 'finished' )(model_concat)


model = tf.keras.Model(inputs = [model_img, tabular], outputs = model_fin)

preprocess = {'featurewise_center': True, 'featurewise_std_normalization' : True}
augment = {'rotation_range' : 30, 'vertical_flip' : True}

# train the model and evaluate it with the test_data
train_and_evaluate(model, image_train, label_train, image_test, label_test, X_train, X_test, preprocess, epochs=50, augment = augment)

This dataset is from a competition in which you have to estimate the 8 images below, these are our scores as of now.

In [9]:
test_images = []
test_names = []

# iterate over the test images
for filename in os.listdir('petfinder-pawpularity-score/test'):
    
    # get the names of the test images and save these in the list
    name = filename.strip('.jpg')
    test_names.append(name)
    
    # save the images 
    img = cv2.imread(os.path.join('petfinder-pawpularity-score/test',filename))
    if img is not None:
        test_images.append(img)

In [10]:
# convert the list of images to an array
test_images = np.array(test_images)

# use the model to predict the pawpularity scores of the test images
model.predict(test_images)

array([[34.538834],
       [36.815617],
       [35.75768 ],
       [34.47796 ],
       [34.48319 ],
       [34.97082 ],
       [34.463966],
       [35.433342]], dtype=float32)